# 女子高校生対象AI活用型データサイエンス教育：統計分析

本ノートブックでは、以下の分析を実施します：
1. データの読み込みと前処理
2. 自己効力感の事前・事後比較（Wilcoxon検定、FDR補正）
3. 効果量の算出（paired Cohen's d）
4. 事後アンケートの記述統計
5. 可視化

## 0. 環境設定

In [ ]:
# ライブラリのインポート
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
try:
    import japanize_matplotlib
    print('✅ 日本語フォント設定完了')
except ImportError:
    print('⚠️ japanize-matplotlib をインストールしてください')

# 表示設定
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

## 1. 定数定義

In [ ]:
# ファイルパス（環境に合わせて変更）
BEFORE_SURVEY_FILE = 'data/sample_before.csv'
AFTER_SURVEY_FILE = 'data/sample_after.csv'

# 共通質問項目（自己効力感4項目）
COMMON_QUESTIONS = [
    'データを表やグラフに整理できる自信がある。',
    'グラフから特徴や傾向を読み取れる自信がある。',
    'プログラムを実行し結果を解釈できる自信がある。',
    '身近なテーマでデータ分析課題を立てられる自信がある。'
]

# 短縮ラベル
SHORT_LABELS = ['Q1 整理', 'Q2 読解', 'Q3 プログラム', 'Q4 課題設定']

# ID列
ID_COLUMN = '匿名コード（8桁の半角英数字）'
EXCLUDE_COLUMNS = ['タイムスタンプ', ID_COLUMN]

## 2. 統計関数の定義

In [ ]:
def cohens_d_paired(before, after):
    """対応のある効果量（paired d）を算出
    
    Parameters
    ----------
    before : array-like
        事前スコア
    after : array-like
        事後スコア
    
    Returns
    -------
    float
        効果量d（差分の平均 / 差分の標準偏差）
    """
    diff = np.array(after) - np.array(before)
    diff = diff[~np.isnan(diff)]
    if len(diff) < 2:
        return np.nan
    return diff.mean() / diff.std(ddof=1)


def benjamini_hochberg_fdr(p_values):
    """Benjamini-Hochberg法によるFDR補正
    
    Parameters
    ----------
    p_values : array-like
        p値の配列（NaNを含んでも可）
    
    Returns
    -------
    array
        FDR補正後のq値
    """
    p = np.array(p_values, dtype=float)
    n = np.sum(~np.isnan(p))
    q = np.full_like(p, np.nan)
    
    if n == 0:
        return q
    
    # NaNでない値のインデックスを取得
    valid_idx = np.where(~np.isnan(p))[0]
    valid_p = p[valid_idx]
    
    # ソート
    sorted_idx = np.argsort(valid_p)
    sorted_p = valid_p[sorted_idx]
    ranks = np.arange(1, n + 1)
    
    # q値の計算
    q_sorted = sorted_p * n / ranks
    
    # 単調性の確保（後ろから累積最小値）
    q_sorted = np.minimum.accumulate(q_sorted[::-1])[::-1]
    q_sorted = np.clip(q_sorted, 0, 1)
    
    # 元の順序に戻す
    q[valid_idx[sorted_idx]] = q_sorted
    
    return q


def interpret_d(d):
    """効果量dの解釈（Cohen, 1988）"""
    if np.isnan(d):
        return '-'
    abs_d = abs(d)
    if abs_d < 0.2:
        return 'ごく小'
    elif abs_d < 0.5:
        return '小'
    elif abs_d < 0.8:
        return '中'
    else:
        return '大'

## 3. データの読み込みと前処理

In [ ]:
def load_and_preprocess(filepath, encodings=['utf-8-sig', 'utf-8', 'shift-jis', 'cp932']):
    """CSVファイルを読み込み、重複を除去"""
    df = None
    for enc in encodings:
        try:
            df = pd.read_csv(filepath, encoding=enc)
            print(f'✅ {filepath} を {enc} で読み込みました')
            break
        except UnicodeDecodeError:
            continue
    
    if df is None:
        raise Exception(f'ファイルを読み込めません: {filepath}')
    
    # 重複処理（タイムスタンプが最新のものを残す）
    if 'タイムスタンプ' in df.columns and ID_COLUMN in df.columns:
        df['タイムスタンプ'] = pd.to_datetime(df['タイムスタンプ'])
        df = df.sort_values(by=[ID_COLUMN, 'タイムスタンプ'], ascending=[True, False])
        df = df.drop_duplicates(subset=[ID_COLUMN], keep='first')
        print(f'   重複除去後: {len(df)}件')
    
    return df

# データ読み込み（ファイルがある場合のみ実行）
# df_before = load_and_preprocess(BEFORE_SURVEY_FILE)
# df_after = load_and_preprocess(AFTER_SURVEY_FILE)

## 4. 事前・事後の対応データ作成

In [ ]:
def create_paired_data(df_before, df_after, questions, id_col):
    """事前・事後の対応データを作成"""
    
    # 必要な列を抽出
    cols = [id_col] + questions
    df_b = df_before[cols].copy()
    df_a = df_after[cols].copy()
    
    # 数値変換
    for q in questions:
        df_b[q] = pd.to_numeric(df_b[q], errors='coerce')
        df_a[q] = pd.to_numeric(df_a[q], errors='coerce')
    
    # リネーム
    df_b = df_b.rename(columns={q: f'{q}_before' for q in questions})
    df_a = df_a.rename(columns={q: f'{q}_after' for q in questions})
    
    # マージ
    df_merged = pd.merge(df_b, df_a, on=id_col, how='inner')
    print(f'対応データ: {len(df_merged)}名')
    
    return df_merged

# df_merged = create_paired_data(df_before, df_after, COMMON_QUESTIONS, ID_COLUMN)

## 5. 統計検定の実施

In [ ]:
def run_paired_tests(df_merged, questions, labels=None):
    """対応のある検定を実施し、結果を表形式で返す"""
    
    if labels is None:
        labels = [f'Q{i+1}' for i in range(len(questions))]
    
    results = []
    p_values = []
    
    for q, label in zip(questions, labels):
        before = df_merged[f'{q}_before'].dropna()
        after = df_merged[f'{q}_after'].dropna()
        
        # 両方に値がある行のみ
        mask = ~(df_merged[f'{q}_before'].isna() | df_merged[f'{q}_after'].isna())
        b = df_merged.loc[mask, f'{q}_before']
        a = df_merged.loc[mask, f'{q}_after']
        n = len(b)
        
        if n < 2:
            results.append({
                '項目': label,
                'n': n,
                'M前(SD)': '-',
                'M後(SD)': '-',
                '変化': '-',
                'd': '-',
                't_p': np.nan,
                'wilcoxon_p': np.nan
            })
            p_values.append(np.nan)
            continue
        
        # 統計量の計算
        mean_b, std_b = b.mean(), b.std()
        mean_a, std_a = a.mean(), a.std()
        change = mean_a - mean_b
        d = cohens_d_paired(b, a)
        
        # t検定
        t_stat, t_p = stats.ttest_rel(a, b)
        
        # Wilcoxon検定
        try:
            w_stat, w_p = stats.wilcoxon(a - b, alternative='two-sided')
        except ValueError:
            w_p = np.nan
        
        results.append({
            '項目': label,
            'n': n,
            'M前(SD)': f'{mean_b:.2f} ({std_b:.2f})',
            'M後(SD)': f'{mean_a:.2f} ({std_a:.2f})',
            '変化': f'{change:+.2f}',
            'd': f'{d:.2f}',
            't_p': t_p,
            'wilcoxon_p': w_p
        })
        p_values.append(w_p)
    
    # FDR補正
    q_values = benjamini_hochberg_fdr(p_values)
    
    for i, q_val in enumerate(q_values):
        if np.isnan(q_val):
            results[i]['wilcoxon_q'] = '-'
        else:
            sig = '**' if q_val < 0.01 else '*' if q_val < 0.05 else ''
            results[i]['wilcoxon_q'] = f'{q_val:.3f}{sig}'
    
    return pd.DataFrame(results)

# 実行例（データがある場合）
# results_df = run_paired_tests(df_merged, COMMON_QUESTIONS, SHORT_LABELS)
# print(results_df.to_string(index=False))

## 6. 可視化

In [ ]:
def plot_pre_post_scatter(df_merged, question, title, save_path=None):
    """事前・事後の散布図を作成"""
    
    before_col = f'{question}_before'
    after_col = f'{question}_after'
    
    plt.figure(figsize=(8, 8))
    
    # 散布図（ジッター付き）
    sns.regplot(
        x=df_merged[before_col], 
        y=df_merged[after_col],
        fit_reg=False, 
        x_jitter=0.2, 
        y_jitter=0.2,
        scatter_kws={'alpha': 0.6, 's': 100}
    )
    
    # 対角線（変化なし）
    plt.plot([1, 5], [1, 5], 'r--', linewidth=2, label='変化なし')
    
    plt.title(title, fontsize=14)
    plt.xlabel('事前の回答 (1:自信がない 〜 5:自信がある)', fontsize=12)
    plt.ylabel('事後の回答 (1:自信がない 〜 5:自信がある)', fontsize=12)
    plt.xticks(range(1, 6))
    plt.yticks(range(1, 6))
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(loc='upper left')
    plt.axis('square')
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f'✅ 保存: {save_path}')
    
    plt.show()


def plot_bar_chart(data, title, xlabel='回答', ylabel='回答者数', save_path=None):
    """棒グラフを作成"""
    
    plt.figure(figsize=(10, 7))
    ax = sns.countplot(data=data, x='label', hue='label', palette='viridis', legend=False)
    
    plt.title(title, fontsize=14)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    
    # 数値ラベル
    for p in ax.patches:
        ax.annotate(
            f'{int(p.get_height())}',
            (p.get_x() + p.get_width() / 2., p.get_height()),
            ha='center', va='center', 
            xytext=(0, 5), textcoords='offset points'
        )
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f'✅ 保存: {save_path}')
    
    plt.show()

## 7. ワードクラウド生成

In [ ]:
def create_wordcloud(texts, font_path=None, save_path=None):
    """ワードクラウドを生成"""
    
    # テキストの結合
    text_data = ' '.join(texts.dropna().astype(str).tolist())
    
    if not text_data.strip():
        print('⚠️ テキストデータがありません')
        return
    
    # ストップワード
    stopwords = {' ', 'こと', 'もの', 'ため', 'これ', 'それ', 'ある', 'いる', 'する', 'なる', 'できる'}
    
    # ワードクラウド生成
    wc_kwargs = {
        'width': 800,
        'height': 400,
        'background_color': 'white',
        'stopwords': stopwords,
        'collocations': False
    }
    
    if font_path:
        wc_kwargs['font_path'] = font_path
    
    wc = WordCloud(**wc_kwargs).generate(text_data)
    
    plt.figure(figsize=(12, 6))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title('自由記述のワードクラウド', fontsize=14)
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f'✅ 保存: {save_path}')
    
    plt.show()

## 8. 内的整合性（Cronbach's α）

In [ ]:
def cronbachs_alpha(df, items):
    """Cronbach's αを算出
    
    Parameters
    ----------
    df : DataFrame
        データフレーム
    items : list
        項目名のリスト
    
    Returns
    -------
    float
        Cronbach's α
    """
    item_data = df[items].dropna()
    n_items = len(items)
    
    if len(item_data) < 2 or n_items < 2:
        return np.nan
    
    # 各項目の分散
    item_vars = item_data.var(ddof=1)
    sum_item_vars = item_vars.sum()
    
    # 合計点の分散
    total_var = item_data.sum(axis=1).var(ddof=1)
    
    # Cronbach's α
    alpha = (n_items / (n_items - 1)) * (1 - sum_item_vars / total_var)
    
    return alpha

# 使用例
# alpha_before = cronbachs_alpha(df_before, COMMON_QUESTIONS)
# alpha_after = cronbachs_alpha(df_after, COMMON_QUESTIONS)
# print(f'事前α = {alpha_before:.2f}, 事後α = {alpha_after:.2f}')

## 9. 実行例（サンプルデータ）

In [ ]:
# サンプルデータで動作確認
np.random.seed(42)
n = 38

# サンプルデータ生成（実際のデータに近い分布）
sample_data = pd.DataFrame({
    'Q1_before': np.random.choice([1,2,3,4,5], n, p=[0.15, 0.35, 0.30, 0.15, 0.05]),
    'Q1_after': np.random.choice([1,2,3,4,5], n, p=[0.05, 0.15, 0.35, 0.30, 0.15]),
    'Q2_before': np.random.choice([1,2,3,4,5], n, p=[0.05, 0.20, 0.35, 0.30, 0.10]),
    'Q2_after': np.random.choice([1,2,3,4,5], n, p=[0.05, 0.15, 0.35, 0.30, 0.15]),
})

# 効果量の計算例
d_q1 = cohens_d_paired(sample_data['Q1_before'], sample_data['Q1_after'])
d_q2 = cohens_d_paired(sample_data['Q2_before'], sample_data['Q2_after'])

print(f'Q1の効果量 d = {d_q1:.2f} ({interpret_d(d_q1)})')
print(f'Q2の効果量 d = {d_q2:.2f} ({interpret_d(d_q2)})')

# Wilcoxon検定
_, p1 = stats.wilcoxon(sample_data['Q1_after'] - sample_data['Q1_before'])
_, p2 = stats.wilcoxon(sample_data['Q2_after'] - sample_data['Q2_before'])

# FDR補正
q_values = benjamini_hochberg_fdr([p1, p2])

print(f'\nWilcoxon検定（FDR補正後）:')
print(f'  Q1: p = {p1:.4f}, q = {q_values[0]:.4f}')
print(f'  Q2: p = {p2:.4f}, q = {q_values[1]:.4f}')

---

## 参考文献

- Bandura, A. (1977). Self-efficacy: Toward a unifying theory of behavioral change. *Psychological Review*, 84(2), 191-215.
- Cohen, J. (1988). *Statistical power analysis for the behavioral sciences* (2nd ed.). Lawrence Erlbaum Associates.
- Benjamini, Y., & Hochberg, Y. (1995). Controlling the false discovery rate: A practical and powerful approach to multiple testing. *Journal of the Royal Statistical Society: Series B*, 57(1), 289-300.